In [1]:
import pandas as pd
from sqlalchemy import create_engine
import requests
import lxml.html as lh
from bs4 import BeautifulSoup
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from config import pw
import time
from splinter import Browser

In [8]:
executable_path = {'executable_path': 'chromedriver (3).exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [9]:
url = 'http://flavorsofcacao.com/usa_craft_makers.html'
browser.visit(url)
time.sleep(2)
html = browser.html
#print(html)
bsoup = soup(html, 'html.parser')
time.sleep(4)
print(bsoup)
results = bsoup.find(id="spryregion2")
results = results.find_all('table')[0]
column_headers = ['Company','City', 'State', 'Owner/Maker']
records = []
for idy, row in enumerate(results.findAll('tr')):
    if idy == 0:
        continue
    cols = row.findAll('td')
    record = {}
    for idx, col in enumerate(cols):
        record[column_headers[idx]] = col.text.strip()
    records.append(record)
browser.quit()
df = pd.DataFrame.from_dict(records)
df.head()

WebDriverException: Message: chrome not reachable
  (Session info: chrome=84.0.4147.125)
